In [ ]:
# Import necessary libraries
import fastf1 as ff1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Initialize maximum values for normalization
max_database_tyre_meters = 0
max_database_distance = 0
max_air_temp = 0
max_humidity = 0
max_track_temp = 0
max_pressure = 0
max_wind_speed = 0

# Get circuit length (in meters)
circuits_length = {
    'Melbourne': 5278,          # Albert Park Circuit
    'Shanghai': 5451,           # Shanghai International Circuit
    'Suzuka': 5807,             # Suzuka International Racing Course
    'Sakhir': 5412,             # Bahrain International Circuit
    'Jeddah': 6174,             # Jeddah Corniche Circuit
    'Miami': 5412,              # Miami International Autodrome
    'Imola': 4909,              # Autodromo Enzo e Dino Ferrari
    'Monte Carlo': 3337,        # Circuit de Monaco
    'Monaco': 3337,             # Circuit de Monaco
    'Barcelona': 4657,          # Circuit de Barcelona-Catalunya
    'Montréal': 4361,           # Circuit Gilles Villeneuve
    'Spielberg': 4318,          # Red Bull Ring (Austria)
    'Silverstone': 5891,        # Silverstone Circuit
    'Spa-Francorchamps': 7004,  # Circuit de Spa-Francorchamps
    'Budapest': 4381,           # Hungaroring
    'Zandvoort': 4259,          # Circuit Zandvoort
    'Monza': 5793,              # Autodromo Nazionale Monza
    'Baku': 6003,               # Baku City Circuit
    'Singapore': 4940,          # Marina Bay Street Circuit
    'Austin': 5513,             # Circuit of the Americas
    'Mexico City': 4304,        # Autódromo Hermanos Rodríguez
    'São Paulo': 4309,          # Autódromo José Carlos Pace
    'Las Vegas': 6201,          # Las Vegas Strip Circuit
    'Lusail': 5419,             # Losail International Circuit
    'Yas Marina': 5281,         # Yas Marina Circuit
    'Yas Island': 5281,         # Yas Marina Circuit
    'Le Castellet': 5842,       # Circuit Paul Ricard
    'Sochi': 5848,              # Sochi Autodrom
    'Portimão': 4653,           # Autódromo Internacional do Algarve
    'Istanbul': 5338,           # Istanbul Park
    'Nürburgring': 5148,        # Nürburgring
    'Hockenheim': 4574,         # Hockenheimring
    'Adelaide': 3780,           # Adelaide Street Circuit
    'Phoenix': 3770,            # Phoenix Street Circuit
    'Estoril': 4415,            # Autódromo do Estoril
    'Magny-Cours': 4411,        # Circuit de Nevers Magny-Cours
    'Indianapolis': 4180,       # Indianapolis Motor Speedway
    'Kuala Lumpur': 5543,       # Sepang International Circuit
    'Valencia': 4005,           # Circuit Ricardo Tormo
    'Yeongam': 5603,            # Korean International Circuit
    'Jerez de la Frontera': 4423,  # Circuit de Jerez
    'Kyalami': 4300,            # Kyalami Grand Prix Circuit
    'Donington': 4023,          # Donington Park Circuit
    'Oyama': 4571,              # Fuji Speedway
    'Mugello': 5245             # Autodromo internazionale del Mugello
}

years = [2018, 2019 ,2020, 2021, 2022, 2023, 2024]  # Years to analyze
all_tire_data = []  # To store all tire data before normalization
all_tire_life_data = []  # To store all tire life data before normalization

# Step 1: Gather all data and find maximum values
for year in years:

    try:
        calendario = ff1.get_event_schedule(year)
        if calendario is None or calendario.empty:
            print(f"No schedule data available for {year}")
            continue
    except Exception as e:
        print(f"Error getting schedule for {year}: {e}")
        continue

    # Filter only conventional races
    gare_ufficiali = calendario[calendario['EventFormat'] == 'conventional']
    lista_gran_premi = gare_ufficiali['EventName'].tolist()
    
    for race in lista_gran_premi:
        # Load the session data for the specified race
        session = ff1.get_session(year, race, 'R')
        session.load()
        location = session.event.Location
        circuit_length = circuits_length.get(location, 0)
        try:
            weather_data = session.weather_data
        except Exception as e:
            print(f"Error getting weather data for {year} {race}: {e}")
            continue

        # Update maximum distance based on total race distance
        total_laps = session.total_laps
        total_meters = total_laps * circuit_length
        max_database_distance = max(max_database_distance, total_meters)

        if session.drivers:
            drivers = session.drivers

            for target_driver in drivers:
                # Get the laps data for the target driver
                driver_laps = session.laps.pick_driver(target_driver)

                # Check if driver_laps is not empty before accessing data
                if not driver_laps.empty:
                    driver_name = driver_laps.iloc[0]['Driver']
                else:
                    continue

                # Track stints to include all of them in the tire life data
                current_stints = {}  # To track current stint info by stint number
                
                # Process lap data
                for lap in driver_laps.iterlaps():
                    lap_number = lap[1]['LapNumber']
                    position = lap[1]['Position']
                    lap_time = lap[1]['LapTime']
                    compound = lap[1]['Compound']
                    
                    if compound == 'SOFT':
                        compound_01 = 1
                    elif compound == 'MEDIUM':
                        compound_01 = 2
                    elif compound == 'HARD':
                        compound_01 = 3
                    else:
                        compound_01 = 0
                    
                    tire_life = lap[1]['TyreLife']
                    fresh_tire = lap[1]['FreshTyre']
                    stint = lap[1]['Stint']
                    lap_start_driver = lap[1]['LapStartTime']
                    lap_finish_driver = lap[1]['Time']

                    fresh_tire = 1 if fresh_tire else 0
                    pitI = 1 if lap[1]['PitInTime'] is not None else 0
                    pitO = 1 if lap[1]['PitOutTime'] is not None else 0
                    best = 1 if lap[1]['IsPersonalBest'] else 0

                    # Get weather data for this lap
                    weather_first_lap = weather_data[
                        (weather_data['Time'] >= lap_start_driver) & 
                        (weather_data['Time'] <= lap_finish_driver)
                    ]
                    
                    l = len(weather_first_lap)
                    
                    # Initialize weather values
                    weather_values = {
                        'air_temp': 0,
                        'humidity': 0,
                        'track_temp': 0,
                        'pressure': 0,
                        'wind_speed': 0,
                        'wind_direction': 0
                    }
                    
                    if l > 0:
                        weather_values['air_temp'] = weather_first_lap['AirTemp'].mean()
                        weather_values['humidity'] = weather_first_lap['Humidity'].mean()
                        weather_values['track_temp'] = weather_first_lap['TrackTemp'].mean()
                        weather_values['pressure'] = weather_first_lap['Pressure'].mean()
                        weather_values['wind_speed'] = weather_first_lap['WindSpeed'].mean()
                        weather_values['wind_direction'] = weather_first_lap['WindDirection'].mean()
                        
                        # Update maximum values for normalization
                        max_air_temp = max(max_air_temp, weather_first_lap['AirTemp'].max())
                        max_humidity = max(max_humidity, weather_first_lap['Humidity'].max())
                        max_track_temp = max(max_track_temp, weather_first_lap['TrackTemp'].max())
                        max_pressure = max(max_pressure, weather_first_lap['Pressure'].max())
                        max_wind_speed = max(max_wind_speed, weather_first_lap['WindSpeed'].max())

                    # Calculate distances
                    if tire_life and tire_life > 0:
                        cumulative_tire_distance = int(tire_life) * circuit_length
                        driver_meters = int(lap_number) * circuit_length
                        
                        # Update global maximum values
                        max_database_tyre_meters = max(max_database_tyre_meters, cumulative_tire_distance)
                        
                        # Store data for later normalization
                        tire_data_entry = {
                            'DriverID': target_driver,
                            'Driver': driver_name,
                            'Team': lap[1]['Team'],
                            'Year': year,
                            'GranPrix': race,
                            'Location': location,
                            'LapNumber': lap_number,
                            'position': position,
                            'PitInTime': pitI,
                            'PitOutTime': pitO,
                            'Stint': stint,
                            'TireLife': tire_life,
                            'TireMeters': cumulative_tire_distance,
                            'Compound': compound_01,
                            'IsFreshTire': fresh_tire,
                            'DriverMeters': driver_meters,
                            'LapTime': lap_time.total_seconds(),
                            'Best': best,
                            'air_temp': weather_values['air_temp'],
                            'humidity': weather_values['humidity'],
                            'trackTemp': weather_values['track_temp'],
                            'pressure': weather_values['pressure'],
                            'wind_speed': weather_values['wind_speed'],
                            'wind_direction': weather_values['wind_direction']
                        }
                        all_tire_data.append(tire_data_entry)
                        
                        # Update current stint information
                        current_stints[stint] = {
                            'DriverID': target_driver,
                            'Driver': driver_name,
                            'Year': year,
                            'GranPrix': race,
                            'Location': location,
                            'Stint': stint,
                            'Compound': compound,
                            'TireLife': tire_life,
                            'TireMeters': cumulative_tire_distance
                        }
                    else:
                        # Add entry even with zero values to maintain complete dataset
                        tire_data_entry = {
                            'DriverID': target_driver,
                            'Driver': driver_name,
                            'Team': lap[1]['Team'],
                            'Year': year,
                            'GranPrix': race,
                            'Location': location,
                            'LapNumber': lap_number,
                            'position': position,
                            'PitInTime': pitI,
                            'PitOutTime': pitO,
                            'Stint': 0,
                            'TireLife': 0,
                            'TireMeters': 0,
                            'Compound': None,
                            'IsFreshTire': fresh_tire,
                            'DriverMeters': 0,
                            'LapTime': lap_time.total_seconds(),
                            'Best': best,
                            'air_temp': weather_values['air_temp'],
                            'humidity': weather_values['humidity'],
                            'trackTemp': weather_values['track_temp'],
                            'pressure': weather_values['pressure'],
                            'wind_speed': weather_values['wind_speed'],
                            'wind_direction': weather_values['wind_direction']
                        }
                        all_tire_data.append(tire_data_entry)

                # After processing all laps, add all stints to tire_life_data
                for stint_data in current_stints.values():
                    all_tire_life_data.append(stint_data)
        else:
            print(f"Session {location} in {year} has no drivers data.")
            continue

# Step 2: Normalize data
for entry in all_tire_data:
    # Normalize distances
    entry['NormalizedTireMeters'] = entry['TireMeters'] / max_database_tyre_meters if max_database_tyre_meters > 0 else 0
    entry['NormalizedDriverMeters'] = entry['DriverMeters'] / max_database_distance if max_database_distance > 0 else 0
    
    # Normalize weather data
    entry['NormalizedAirTemp'] = entry['air_temp'] / max_air_temp if max_air_temp > 0 else 0
    entry['NormalizedHumidity'] = entry['humidity'] / max_humidity if max_humidity > 0 else 0
    entry['NormalizedTrackTemp'] = entry['trackTemp'] / max_track_temp if max_track_temp > 0 else 0
    entry['NormalizedPressure'] = entry['pressure'] / max_pressure if max_pressure > 0 else 0
    entry['NormalizedWindSpeed'] = entry['wind_speed'] / max_wind_speed if max_wind_speed > 0 else 0
    entry['NormalizedWindDirection'] = entry['wind_direction'] / 360  # Wind direction is already 0-360

for entry in all_tire_life_data:
    entry['NormalizedTireMeters'] = entry['TireMeters'] / max_database_tyre_meters if max_database_tyre_meters > 0 else 0

# Step 3: Save to CSV files
driver_race_data = {}  # To group data by driver and race

# Group data by driver and race for easier CSV writing
for entry in all_tire_data:
    key = (entry['DriverID'], entry['GranPrix'], entry['Year'])
    if key not in driver_race_data:
        driver_race_data[key] = []
    driver_race_data[key].append(entry)

driver_race_life_data = {}  # Group tire life data similarly
for entry in all_tire_life_data:
    key = (entry['DriverID'], entry['GranPrix'], entry['Year'])
    if key not in driver_race_life_data:
        driver_race_life_data[key] = []
    driver_race_life_data[key].append(entry)

#Save data to CSVs
for (driver, race, year), data in driver_race_data.items():
    location = data[0]['Location']  # Get location from first entry
    df = pd.DataFrame(data)
    filename = f'{driver}_{race}_location_{location}_{year}_all_tire_driver_race_data.csv'
    df.to_csv(filename, index=False)

for (driver, race, year), data in driver_race_life_data.items():
    location = data[0]['Location']  # Get location from first entry
    df = pd.DataFrame(data)
    filename = f'{driver}_{race}_lives_{location}_{year}_tire_live_data.csv'
    df.to_csv(filename, index=False)

print("\nMaximum values for normalization:")
print(f"Max tire meters: {max_database_tyre_meters}")
print(f"Max distance: {max_database_distance}")
print(f"Max air temperature: {max_air_temp}°C")
print(f"Max humidity: {max_humidity}%")
print(f"Max track temperature: {max_track_temp}°C")
print(f"Max pressure: {max_pressure}hPa")
print(f"Max wind speed: {max_wind_speed}km/h")
print("\nAll data has been normalized and saved!")

req         WARNING 	DEFAULT CACHE ENABLED! (9.49 GB) C:\Users\dilau\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Australian Grand Prix - Race [v3.4.5]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INF

Error getting weather data for 2022 Canadian Grand Prix: The data you are trying to access has not been loaded yet. See `Session.load`


core        WARNING 	Failed to load extended driver information!
Request for URL https://ergast.com/api/f1/2022/10/results.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\requests_cache\session.py", line 286, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\requests_cache\session.py", line 254, in _send_and_cache
    response = super().send(request, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\req.py", line 134, in send
    lim.limit()
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\req.py", line 104, in limit
    raise RateLimitExceededError(self._info)
fastf1.req.RateLimitExc

Error getting weather data for 2022 British Grand Prix: The data you are trying to access has not been loaded yet. See `Session.load`


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
Request for URL https://ergast.com/api/f1/2022/12/results.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\requests_cache\session.py", line 286, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\requests_cache\session.py", line 254, in _send_and_cache
    response = super().send(request, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\req.py", line 134, in send
    lim.limit()
 

...

Error getting weather data for 2022 Abu Dhabi Grand Prix: The data you are trying to access has not been loaded yet. See `Session.load`


logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


Error getting schedule for 2023: Failed to load any schedule data.


logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


Error getting schedule for 2024: Failed to load any schedule data.

Maximum values for normalization:
Max tire meters: 304266
Max distance: 470844
Max air temperature: 37.2°C
Max humidity: 97.5%
Max track temperature: 57.2°C
Max pressure: 1023.3hPa
Max wind speed: 7.9km/h

All data has been normalized and saved!
